<a href="https://colab.research.google.com/github/sharathshanmukh/Recommendations-systems/blob/main/movie_lens(collabrative).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import pandas as pd

In [2]:
import zipfile
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip --no-check-certificate


--2021-05-23 13:27:49--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  15.5MB/s    in 0.3s    

2021-05-23 13:27:50 (15.5 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [3]:
zipfile.ZipFile("/content/ml-100k.zip").extractall()

In [4]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('/content/ml-100k/u.data', sep='\t', names=column_names)

In [5]:
df

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [ ]:
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('/content/ml-100k/u.item', sep='|', names=m_cols, usecols=range(5),
                     encoding='latin-1').rename(columns={"movie_id":"item_id"})
movies

,item_id,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)
...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998)
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...


In [ ]:
movie=movies[["item_id","title"]]
movie

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [ ]:
df=pd.merge( df,movie,how="left",on="item_id")

In [ ]:
df['title'].nunique()

1664

In [ ]:
n_reviews=df.groupby('title').size().sort_values(ascending=False).reset_index().rename(columns={0:"number_of_reviews"})

In [ ]:
df.groupby('title')['rating'].mean().sort_values(ascending=False)

title
Marlene Dietrich: Shadow and Light (1996)       5.0
Prefontaine (1997)                              5.0
Santa with Muscles (1996)                       5.0
Star Kid (1997)                                 5.0
Someone Else's America (1995)                   5.0
                                               ... 
Touki Bouki (Journey of the Hyena) (1973)       1.0
JLG/JLG - autoportrait de décembre (1994)       1.0
Daens (1992)                                    1.0
Butterfly Kiss (1995)                           1.0
Eye of Vichy, The (Oeil de Vichy, L') (1993)    1.0
Name: rating, Length: 1664, dtype: float64

In [ ]:
df

,user_id,item_id,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,186,302,3,891717742,L.A. Confidential (1997)
2,22,377,1,878887116,Heavyweights (1994)
3,244,51,2,880606923,Legends of the Fall (1994)
4,166,346,1,886397596,Jackie Brown (1997)
...,...,...,...,...,...
99995,880,476,3,880175444,"First Wives Club, The (1996)"
99996,716,204,5,879795543,Back to the Future (1985)
99997,276,1090,1,874795795,Sliver (1993)
99998,13,225,2,882399156,101 Dalmatians (1996)


In [ ]:
moviemat = df.pivot_table(index='user_id',columns='title',values='rating')


In [ ]:
toystory_corr=moviemat.corrwith(moviemat["Toy Story (1995)"]).dropna().reset_index().rename(columns={0:"correlation"}).sort_values(by="correlation",ascending=False)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [ ]:
toystory_corr.join(n_reviews['number_of_reviews'])


,title,correlation,number_of_reviews
909,"Old Lady Who Walked in the Sea, The (Vieille q...",1.0,21
1020,Reckless (1995),1.0,15
696,Ladybird Ladybird (1994),1.0,40
628,Infinity (1996),1.0,46
35,Albino Alligator (1996),1.0,298
...,...,...,...
1118,"Slingshot, The (1993)",-1.0,10
563,Heavy (1995),-1.0,57
423,Feast of July (1995),-1.0,79
1151,Stalker (1979),-1.0,9


In [ ]:
ratings = pd.DataFrame(df.groupby('title')['rating'].mean())
ratings['num of ratings'] = pd.DataFrame(df.groupby('title')['rating'].count())


In [ ]:
toystory_corr.join(ratings["num of ratings"],on="title")


,title,correlation,num of ratings
909,"Old Lady Who Walked in the Sea, The (Vieille q...",1.0,5
1020,Reckless (1995),1.0,8
696,Ladybird Ladybird (1994),1.0,4
628,Infinity (1996),1.0,6
35,Albino Alligator (1996),1.0,6
...,...,...,...
1118,"Slingshot, The (1993)",-1.0,2
563,Heavy (1995),-1.0,5
423,Feast of July (1995),-1.0,5
1151,Stalker (1979),-1.0,11


In [ ]:
ratings.loc['Reckless (1995)']

rating            2.75
num of ratings    8.00
Name: Reckless (1995), dtype: float64

In [ ]:
n_reviews[n_reviews["title"]=='Albino Alligator (1996)']

,title,number_of_reviews
1259,Albino Alligator (1996),6


In [ ]:
ratings

,rating,num of ratings
title,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5
101 Dalmatians (1996),2.908257,109
12 Angry Men (1957),4.344000,125
187 (1997),3.024390,41
...,...,...
Young Guns II (1990),2.772727,44
"Young Poisoner's Handbook, The (1995)",3.341463,41
Zeus and Roxanne (1997),2.166667,6


In [ ]:
filtered_recomenders=pd.merge(toystory_corr,n_reviews,on="title",how="left")

In [ ]:
filtered_recomenders[filtered_recomenders['number_of_reviews']>100].sort_values(by=)

,title,correlation,number_of_reviews
5,Toy Story (1995),1.000000,452
147,"Craft, The (1996)",0.549100,104
214,Down Periscope (1996),0.457995,101
218,Miracle on 34th Street (1994),0.456291,101
219,G.I. Jane (1997),0.454756,175
...,...,...,...
1190,Raging Bull (1980),-0.083523,116
1192,Boogie Nights (1997),-0.086194,189
1196,"Clockwork Orange, A (1971)",-0.087710,221
1207,Apt Pupil (1998),-0.104066,160


In [ ]:
url='https://files.grouplens.org/datasets/movielens/ml-100k.zip'

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
# !unzip ml-100k.zip

--2021-05-23 13:10:49--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
ERROR: cannot verify files.grouplens.org's certificate, issued by ‘CN=InCommon RSA Server CA,OU=InCommon,O=Internet2,L=Ann Arbor,ST=MI,C=US’:
  Unable to locally verify the issuer's authority.
To connect to files.grouplens.org insecurely, use `--no-check-certificate'.
